In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [2]:
import openml
import os
import 
import pandas as pd
import math

In [4]:
import sys

sys.path.append("../..")
from src.load_datasets import load_dataset, load_rankings, load_train_data

# Load data

In [5]:
dataset = load_dataset('../../data/raw/dataset.csv')
rankings = load_rankings('../../data/raw/rankings.csv')
X_train, y_train = load_train_data('../../data/raw/dataset_train.csv')

Loading data ...
Loading rankings ...
Loading train data ...


In [8]:
dataset_agg = pd.read_csv('../../data/preprocessed/dataset_agg.csv', index_col=0)

In [9]:
dataset_agg.head()

,Quartile1KurtosisOfNumericAtts,J48.001.ErrRate,RandomTreeDepth3ErrRate,Quartile2KurtosisOfNumericAtts,Dimensionality,J48.00001.ErrRate,Quartile2MutualInformation,MinSkewnessOfNumericAtts,Quartile2AttributeEntropy,MinorityClassSize,...,non_categorical_features_count,ratio_of_categorical_features_to_all,sum_of_all_categories,categorical_target_variables_count,non_categorical_target_variables_count,categorical_target_values_sum,total_feature_count,min_number_of_categories_per_cat_feature,max_number_of_categories_per_cat_feature,avg_number_of_categories_per_cat_feature
0,NaN,0.007822,0.054443,NaN,0.011577,0.007822,0.005426,NaN,0.679500,1527.0,...,0,1.000000,74,1,0,2.0,36,2,3,2.055556
1,1.985325,0.160870,0.249275,15.348781,0.023188,0.160870,0.029603,1.145892,0.984795,307.0,...,6,0.600000,41,1,0,2.0,15,2,14,4.555556
2,-1.210473,0.279000,0.298000,0.919781,0.021000,0.279000,0.012753,-0.531348,1.532104,300.0,...,7,0.650000,56,1,0,2.0,20,2,11,4.307692
3,5.982197,0.015642,0.036585,8.871304,0.007953,0.015642,0.001290,1.232674,0.259375,231.0,...,7,0.758621,46,1,0,2.0,29,1,5,2.090909
4,NaN,0.185804,0.229645,NaN,0.010438,0.185804,0.013557,NaN,1.528146,332.0,...,0,1.000000,27,1,0,2.0,9,3,3,3.000000


In [33]:
# Drop columns with more nan values than <threshold> (percentage)
nan_threshold = 0.4
replacing_strategy = "mean"  # Valid options are mean or median

# Get column names where the percentage of nan values is greater than the threshold
total_number_of_rows = dataset_agg.shape[0]
sum_of_nan_values = dataset_agg.isna().sum()
ratio_of_nan_values = sum_of_nan_values / total_number_of_rows
selector_of_columns = ratio_of_nan_values[(ratio_of_nan_values > 0.4)]
column_names_to_drop = list(selector_of_columns.index)

# Drop the columns
dataset_agg_filtered = dataset_agg.drop(column_names_to_drop, axis=1)

# Fill the remaining nan values with the mean or median
#dataset_agg_filtered = dataset_agg_filtered.fillna(dataset_agg_filtered.mean())

In [28]:
len(dataset_agg_filtered.columns)

51

In [31]:
sum(dataset_agg_filtered.isna().sum())

285

In [32]:
dataset_agg_filtered = dataset_agg_filtered.fillna(dataset_agg_filtered.mean())
sum(dataset_agg_filtered.isna().sum())

0

### Test merging

In [42]:
X_train.head()

,dataset,model,tuning,scoring,encoder
0,1169,KNC,model,ACC,BUCV2RGLMME
1,1169,KNC,model,ACC,BUCV2TE
2,1169,KNC,model,ACC,CBE
3,1169,KNC,model,ACC,CE
4,1169,KNC,model,ACC,CV10RGLMME


In [43]:
dataset_agg_filtered[dataset_agg_filtered.dataset_id == 1169]

,Dimensionality,Quartile2MutualInformation,Quartile2AttributeEntropy,MinorityClassSize,MajorityClassPercentage,NumberOfBinaryFeatures,Quartile1MutualInformation,EquivalentNumberOfAtts,MaxMutualInformation,MinAttributeEntropy,...,non_categorical_features_count,ratio_of_categorical_features_to_all,sum_of_all_categories,categorical_target_variables_count,non_categorical_target_variables_count,categorical_target_values_sum,total_feature_count,min_number_of_categories_per_cat_feature,max_number_of_categories_per_cat_feature,avg_number_of_categories_per_cat_feature
19,0.000015,0.022126,5.152686,240264.0,55.455771,1.0,0.006293,40.586564,0.051873,2.792166,...,3,0.571429,611,1,0,2.0,7,7,293,152.75


In [45]:
"dataset_id" in dataset_agg_filtered.columns

True

In [47]:
dataset_agg_filtered = dataset_agg_filtered.rename(columns={"dataset_id": "dataset"})
merged_df = X_train.merge(dataset_agg_filtered, how="left", on=["dataset"])
merged_df.head()

,dataset,model,tuning,scoring,encoder,Dimensionality,Quartile2MutualInformation,Quartile2AttributeEntropy,MinorityClassSize,MajorityClassPercentage,...,non_categorical_features_count,ratio_of_categorical_features_to_all,sum_of_all_categories,categorical_target_variables_count,non_categorical_target_variables_count,categorical_target_values_sum,total_feature_count,min_number_of_categories_per_cat_feature,max_number_of_categories_per_cat_feature,avg_number_of_categories_per_cat_feature
0,1169,KNC,model,ACC,BUCV2RGLMME,0.000015,0.022126,5.152686,240264.0,55.455771,...,3,0.571429,611,1,0,2.0,7,7,293,152.75
1,1169,KNC,model,ACC,BUCV2TE,0.000015,0.022126,5.152686,240264.0,55.455771,...,3,0.571429,611,1,0,2.0,7,7,293,152.75
2,1169,KNC,model,ACC,CBE,0.000015,0.022126,5.152686,240264.0,55.455771,...,3,0.571429,611,1,0,2.0,7,7,293,152.75
3,1169,KNC,model,ACC,CE,0.000015,0.022126,5.152686,240264.0,55.455771,...,3,0.571429,611,1,0,2.0,7,7,293,152.75
4,1169,KNC,model,ACC,CV10RGLMME,0.000015,0.022126,5.152686,240264.0,55.455771,...,3,0.571429,611,1,0,2.0,7,7,293,152.75


## Define function

In [50]:
def add_dataset_meta_information(df=None, path_to_meta_df=None, nan_threshold=0.4, replacing_strategy="mean"):
    """
    Adds the meta information to the dataset. 
    The meta dataset contains null values, since not all datasets contain all meta information. 
    Therefore, one can specify a threshold. If a column has more than threshold % of nan values it will be dropped. 
    The remaining nan values can be filled with either the 'mean', 'median' or a given constant.

            Parameters:
                    df (pandas.DataFrame): The basic dataframe to which the meta information will be added. 
                    
                    path_to_meta_df (string): The path from which the meta dataframe will be loaded
                    
                    nan_threshold (float): Has to be in the range of [0, 1]. Columns, which have more nan values than nan_threshold % will be dropped. 
                    
                    replacing_strategy ("mean", "median" or float): The strategy how to replace the reamining nan values. 

            Returns:
                    df (pandas.DataFrame): The dataframe df, which is extended by the meta information. The datasets are merged based on the dataset_id. 
    """
    # Validate option
    assert isinstance(replacing_strategy, float) or isinstance(replacing_strategy, int) or replacing_strategy in ['mean', 'median']
    assert 0 <= nan_threshold <= 1
    assert os.path.exists(path_to_meta_df)
    assert "dataset" in df.columns
    
    # Load the meta dataset
    dataset_agg = pd.read_csv('../../data/preprocessed/dataset_agg.csv', index_col=0)
    
    
    # Get column names where the percentage of nan values is greater than the threshold
    total_number_of_rows = dataset_agg.shape[0]
    sum_of_nan_values = dataset_agg.isna().sum()
    ratio_of_nan_values = sum_of_nan_values / total_number_of_rows
    selector_of_columns = ratio_of_nan_values[(ratio_of_nan_values > nan_threshold)]
    column_names_to_drop = list(selector_of_columns.index)

    # Drop the columns
    dataset_agg_filtered = dataset_agg.drop(column_names_to_drop, axis=1)
    
    # Fill the remaining nan values with the mean or median
    if replacing_strategy == "mean":
        dataset_agg_filtered = dataset_agg_filtered.fillna(dataset_agg_filtered.mean())
    elif replacing_strategy == "median":
        dataset_agg_filtered = dataset_agg_filtered.fillna(dataset_agg_filtered.median())
    else:
        # Only float or integer is possible
        dataset_agg_filtered = dataset_agg_filtered.fillna(replacing_strategy)
    
    # Merge datasets
    dataset_agg_filtered = dataset_agg_filtered.rename(columns={"dataset_id": "dataset"})
    merged_df = df.merge(dataset_agg_filtered, how="left", on=["dataset"])
    
    return merged_df

In [51]:
df = add_dataset_meta_information(df=X_train, path_to_meta_df="../../data/preprocessed/dataset_agg.csv", nan_threshold=0.4, replacing_strategy="mean")

In [52]:
df.head()

,dataset,model,tuning,scoring,encoder,Dimensionality,Quartile2MutualInformation,Quartile2AttributeEntropy,MinorityClassSize,MajorityClassPercentage,...,non_categorical_features_count,ratio_of_categorical_features_to_all,sum_of_all_categories,categorical_target_variables_count,non_categorical_target_variables_count,categorical_target_values_sum,total_feature_count,min_number_of_categories_per_cat_feature,max_number_of_categories_per_cat_feature,avg_number_of_categories_per_cat_feature
0,1169,KNC,model,ACC,BUCV2RGLMME,0.000015,0.022126,5.152686,240264.0,55.455771,...,3,0.571429,611,1,0,2.0,7,7,293,152.75
1,1169,KNC,model,ACC,BUCV2TE,0.000015,0.022126,5.152686,240264.0,55.455771,...,3,0.571429,611,1,0,2.0,7,7,293,152.75
2,1169,KNC,model,ACC,CBE,0.000015,0.022126,5.152686,240264.0,55.455771,...,3,0.571429,611,1,0,2.0,7,7,293,152.75
3,1169,KNC,model,ACC,CE,0.000015,0.022126,5.152686,240264.0,55.455771,...,3,0.571429,611,1,0,2.0,7,7,293,152.75
4,1169,KNC,model,ACC,CV10RGLMME,0.000015,0.022126,5.152686,240264.0,55.455771,...,3,0.571429,611,1,0,2.0,7,7,293,152.75


In [53]:
df.shape

(36054, 55)

In [54]:
X_train.shape

(36054, 5)